# DESCRIPTIVE ANALYSIS (ANÁLISIS DESCRIPTIVO)

El objetivo de esta sección es analizar de la distribución de las variables cómo se relacionan las mismas.

<br>

Se encarga de responder preguntas como:
- ¿Cuál es el valor más frecuente?
- ¿Cómo es la distribución de los datos (normal, tirada a la derecha/izquierda, etc)?
- ¿Cómo se relacionan las variables?

<br>

---

## Configuración General

1. Carga de librerías.
2. Seteo de estilos del notebook.
3. Ingesta del dataset.

In [ ]:
import sys
import os
import statistics

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt



sys.path.append(os.path.abspath(os.path.join('..', '..', 'src', 'utils')))
import utils as ut

In [ ]:
# Seteo de estilos
plt.style.use("ggplot")
sns.set_palette("viridis")
plt.rcParams["figure.figsize"] = (9,6)

In [ ]:
wines = pd.read_csv("../../src/data/transformed/wines_clean.csv")
pd.set_option('display.max_columns', None)
wines.head(3)

<br>
<br>
<br>
<br>
<br>
<br>

---

## 01 | Análisis de Datos Numéricos

Esta sección se enfoca en analizar la frecuencia y distribución de variables numéricas para comprender valores habituales y atípicos, densidad de valores y estadísticas descriptivas generales.

### Rating
- ¿Cuales son los ratings más comunes?
- ¿El rating se relaciona con la cantidad de ratings?
- ¿El rating es un buen parámetro para la recomendación de vinos?
- ¿La cantidad de ratings se relaciona con un buen rating?
<br>
<br>
> Rating: los valores más frecuentes están entre *3.9 y 4.1* aproximadamente, con una distribución más o menos normal.

> Rating Quantity: la distribución está tirada hacia la izquierda, con algunos outliers muy alejados en el extremo derecho. En general, los valores más frecuentes son hasta *160-170* ratings aproximadamente.
<br>
---

In [ ]:
# Métricas generales del rating y rating quantity
wines[["rating", "rating_qty"]].describe().T

In [ ]:
# Distribución del rating
sns.histplot(wines['rating'], bins=20, kde=True)
plt.title('Distribución de Ratings')

plt.tight_layout()
plt.show()

In [ ]:
# Distribución del rating eliminando outliers
no_outlier_rating = ut.manage_outlier_IQR(df=wines["rating"], func="remove")
sns.histplot(no_outlier_rating, binwidth=0.1, kde=True)

In [ ]:
# Distribución de la cantidad de ratings
sns.histplot(wines, x="rating_qty", binwidth=500, kde=True)

In [ ]:
# Distribución de la cantidad de ratings sin outliers
no_outlier_rating_qty = ut.manage_outlier_IQR(df=wines["rating_qty"], func="remove")
sns.histplot(no_outlier_rating_qty, kde=True)

### Price

- ¿Qué sería un vino caro y uno barato?
- ¿Nuestro dataset contiene muchos vinos caros o más baratos?
- ¿Cual es la distribución del precio-calidad de nuestros vinos?
<br>
<br>
---

In [ ]:
# Estadísticas descriptivas del precio
pd.DataFrame(wines["price"]).describe().T

In [ ]:
# Distribución de precios
sns.histplot(wines['price'], bins=30, kde=True)
plt.title('Distribución de Precios')
plt.show()

In [ ]:
# Distribución del precio sin outliers
no_outlier_price = ut.manage_outlier_IQR(df=wines["price"], func="remove")
sns.histplot(no_outlier_price, kde=True)

In [ ]:
# Categorización de vinos en base al precio, según el cuartil de precios sin outliers
cheap_threshold = no_outlier_price.quantile(0.25)
mid_threshold = no_outlier_price.quantile(0.5)
expensive_threshold = no_outlier_price.quantile(0.75)

def categorizar_precio(p, cheap, mid, exp):
    if p <= cheap_threshold:
        return "Barato"
    elif ((p > cheap) & (p <= mid)):
        return "Medio"
    elif ((p > mid) & (p <= exp)):
        return "Medio-Caro"
    elif p > exp:
        return "Caro"
    else:
        return "Desconocido"

wines['Categoria_Precio'] = wines['price'].apply(
    lambda p: categorizar_precio(p, cheap_threshold, mid_threshold, expensive_threshold)
)

categorias_precio = ['Barato', 'Medio', 'Medio-Caro', 'Caro']

categories_map = {
    "Barato": f"0 - {round(cheap_threshold,2)}",
    "Medio": f"{round(cheap_threshold,2)} - {round(mid_threshold,2)}",
    "Medio-Caro": f"{round(mid_threshold,2)} - {round(expensive_threshold,2)}",
    "Caro": f"{round(expensive_threshold,2)}+"
}

ax = sns.countplot(
    data=wines,
    x='Categoria_Precio',
    order=categorias_precio,
)

ax.set_ylim(0, 800)

for p, categoria in zip(ax.patches, categorias_precio):
    height = p.get_height()
    price_range = categories_map[categoria]

    ax.text(
        p.get_x() + p.get_width() / 2,
        height + 2, 
        f'# Vinos: {int(height)}\n$ Precio: {price_range}',
        ha='center',
        va='bottom',
        fontsize=10
    )

plt.title("Cantidad de vinos por categoría de precio")
plt.show()

In [ ]:
# Categorización de vinos según thresholds redondeados, cercanos a cuartiles de outliers (empírica)
cheap_threshold = 18
mid_threshold = 24
expensive_threshold = 35

wines['Categoria_Precio'] = wines['price'].apply(
    lambda p: categorizar_precio(p, cheap_threshold, mid_threshold, expensive_threshold)
)

categorias_precio = ['Barato', 'Medio', 'Medio-Caro', 'Caro']

categories_map = {
    "Barato": f"0 - {round(cheap_threshold,2)}",
    "Medio": f"{round(cheap_threshold,2)} - {round(mid_threshold,2)}",
    "Medio-Caro": f"{round(mid_threshold,2)} - {round(expensive_threshold,2)}",
    "Caro": f"{round(expensive_threshold,2)}+"
}

ax = sns.countplot(
    data=wines,
    x='Categoria_Precio',
    order=categorias_precio,
)

ax.set_ylim(0, 900)

for p, categoria in zip(ax.patches, categorias_precio):
    height = p.get_height()
    price_range = categories_map[categoria]

    ax.text(
        p.get_x() + p.get_width() / 2,
        height + 2, 
        f'# Vinos: {int(height)}\n$ Precio: {price_range}',
        ha='center',
        va='bottom',
        fontsize=10
    )

plt.title("Cantidad de vinos por categoría de precio (empírica)")
plt.show()

In [ ]:
scaler = MinMaxScaler()

wines[["rating_scaled", "price_scaled"]] = scaler.fit_transform(wines[["rating", "price"]])

wines['price_quality_scaled'] = wines['rating_scaled'] / wines['price_scaled']
wines[['rating', 'price', 'price_quality_scaled']].sort_values(by='price_quality_scaled', ascending=False).head(10)
sns.histplot(wines['price_quality_scaled'], kde=True)
plt.title("Distribución de la relación Calidad/Precio")
plt.show()


In [ ]:
wines_filtrados = wines[wines['price'] > 0].copy()
#wines_filtrados['price_quality_scaled'] = wines_filtrados['rating_scaled'] / wines_filtrados['price_scaled']
top_vinos_cp = wines_filtrados.sort_values(by='price_quality_scaled', ascending=False).head(10)
top_vinos_cp[['name', 'price', 'rating', 'rating_scaled', 'price_scaled', 'price_quality_scaled']]


In [ ]:
wines_filtrados['log_price'] = np.log1p(wines_filtrados['price'])
wines_filtrados['calidad_precio_log'] = wines_filtrados['rating'] / wines_filtrados['log_price']
top_vinos_cp_log = wines_filtrados.sort_values(by='calidad_precio_log', ascending=False).head(20)
top_vinos_cp_log[['name', 'price', 'rating', 'log_price', 'calidad_precio_log']]


In [ ]:
sns.histplot(wines_filtrados['calidad_precio_log'], kde=True)
plt.title("Distribución de la relación Calidad/Precio (log) ")
plt.show()

In [ ]:
top10 = top_vinos_cp_log.head(10)

plt.figure(figsize=(10,6))
sns.barplot(data=top10, x="calidad_precio_log", y="name", palette="viridis")
plt.title("Top 10 vinos con mejor relación Calidad/Precio")
plt.xlabel("Calidad / Precio (rating / log(price))")
plt.ylabel("Nombre del vino")
plt.tight_layout()
plt.show()


<br>
<br>
<br>
<br>
<br>
<br>

---

## 02 | Análisis de Variables Categóricas

Esta sección se enfoca en analizar la frecuencia y distribución de variables categóricas para comprender valores habituales y atípicos, densidad de valores y estadísticas descriptivas generales.

### Notas de Sabor

---

In [ ]:
# Corregir. Si le cambiamos los datos esto puede pasar a estar mal.

notes_cols = [
    'black fruit', 'citrus', 'dried fruit', 'earthy', 'floral', 'oaky',
    'red fruit', 'spices', 'tree fruit', 'tropical', 'vegetal', 'yeasty'
]
note_means = wines[notes_cols].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=note_means.values, y=note_means.index, palette="viridis")
plt.title("Notas de sabor más frecuentes en el dataset")
plt.xlabel("Promedio (%) por vino")
plt.ylabel("Nota de sabor")
plt.show()



### Grapes

---

In [ ]:
grape_cols = [
    'Albariño', 'Barbera', 'Bonarda', 'Béquignol Noir', 'Cabernet Franc', 'Cabernet Sauvignon',
    'Cereza', 'Chardonnay', 'Chenin Blanc', 'Criolla Grande', 'Garnacha', 'Gewürztraminer',
    'Grenache', 'Grüner Veltliner', 'Malbec', 'Malvasia', 'Marsanne', 'Mencia', 'Merlot',
    'Moscatel', 'Mourvedre', 'Pais', 'Pedro Ximenez', 'Petit Verdot',
    'Pinot Gris', 'Pinot Noir', 'Riesling', 'Roussanne', 'Sangiovese', 'Sauvignon Blanc',
    'Shiraz/Syrah', 'Sémillon', 'Tannat', 'Tempranillo', 'Torrontés', 'Trousseau',
    'Verdejo', 'Viognier'
]
grape_counts = wines[grape_cols].sum().sort_values(ascending=False)
sns.barplot(x=grape_counts.values, y=grape_counts.index)
plt.title("Cantidad de vinos por variedad de uva")
plt.xlabel("Cantidad de vinos")
plt.ylabel("Variedad de uva")
plt.tight_layout()
plt.show()


### Maridajes

---

In [ ]:
pairing_cols = [
    'any junk food will do', 'aperitif', 'appetizers and snacks', 'beef', 'blue cheese',
    'cured meat', 'game (deer, venison)', "goat's milk cheese", 'lamb', 'lean fish',
    'mature and hard cheese', 'mild and soft cheese', 'mushrooms', 'pasta', 'pork',
    'poultry', 'rich fish (salmon, tuna etc)', 'shellfish', 'spicy food', 'veal', 'vegetarian'
]

pairing_freq = wines_filtrados[pairing_cols].sum().sort_values(ascending=False)

top_pairing = pairing_freq.head(12)

sns.barplot(x=top_pairing.values, y=top_pairing.index)
plt.title("Maridajes más frecuentes en los vinos")
plt.xlabel("Cantidad de vinos")
plt.ylabel("Maridaje")
plt.tight_layout()
plt.show()


### Year (Año)

---

In [ ]:
wines_filtrados.select_dtypes(include=['object']).columns


In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=wines_filtrados, x='year', order=wines_filtrados['year'].value_counts().index[:15])
plt.title("Cantidad de vinos por año ")
plt.xticks(rotation=45)
plt.show()


### Winery (Bodega)

---

In [ ]:
top_bodegas = wines_filtrados['winery'].value_counts().head(10)

sns.barplot(x=top_bodegas.values, y=top_bodegas.index)
plt.title("Bodegas con más vinos registrados")
plt.xlabel("Cantidad de vinos")
plt.ylabel("Bodega")
plt.show()


### Regions (Regiones)

---

In [ ]:
region_cols = [
    'Cafayate Valley', 'Calchaqui Valley', 'Campanha',
    'Famatina', 'Gualtallary', 'La Consulta', 'La Rioja', 'Las Compuertas',
    'Lujan de Cuyo', 'Lunlunta', 'Maipu', 'Mendoza', 'Paraje Altamira',
    'Patagonia', 'Pedernal Valley', 'Perdriel', 'Rio Grande do Sul', 'Rio Negro',
    'Salta', 'San Carlos', 'San Juan', 'San Rafael', 'Serra Gaúcha',
    'Tulum Valley', 'Tunuyán', 'Tupungato', 'Uco Valley', 'Vale dos Vinhedos',
    'Vista Flores'
]

regiones_melted = wines.melt(
    value_vars=region_cols,
    var_name='region',
    value_name='is_region'
)

regiones_validas = regiones_melted[regiones_melted['is_region'] == 1]
conteo_regiones = regiones_validas['region'].value_counts().head(10)

sns.barplot(x=conteo_regiones.values, y=conteo_regiones.index)
plt.title("Regiones más frecuentes productoras de vinos")
plt.xlabel("Cantidad de vinos")
plt.ylabel("Región")
plt.show()



### Wine Style (Estilo de Vino)

---

In [ ]:
style_counts = wines['style'].value_counts().head(10)

sns.set_style("whitegrid")
sns.set_palette("coolwarm")

plt.figure(figsize=(10, 6))
sns.barplot(x=style_counts.values, y=style_counts.index)
plt.title("Estilos de vino más frecuentes")
plt.xlabel("Cantidad de vinos")
plt.ylabel("Estilo")
plt.show()


<br>
<br>
<br>
<br>
<br>

---

## 03 | Cruce de Variables

Esta sección se enfoca en el cruce de dos o tres variables, tanto categóricas como numéricas, para ver frecuencias combinadas y las características de ciertas variables más frecuentes.

### Rating + Rating Quantity

---

In [ ]:
sns.jointplot(data=wines, x="rating", y="rating_qty")

In [ ]:
g = sns.JointGrid(data=wines, x="rating", y="rating_qty")
g.plot_joint(sns.histplot)
g.plot_marginals(sns.boxplot)

In [ ]:

sns.boxplot(data=wines, x='Categoria_Precio', y='rating', order=categorias_precio)
plt.title("Rating según categoría de precio")
plt.show()

wines.groupby('Categoria_Precio')['rating'].mean()


### Regiones + Rating

---

In [ ]:
wines_filtrados_reset = wines_filtrados.reset_index().copy()

regiones_melted = wines_filtrados_reset.melt(
    id_vars=['index', 'rating'], 
    value_vars=region_cols,
    var_name='region',
    value_name='is_region'
)

regiones_validas = regiones_melted[regiones_melted['is_region'] == 1]
region_rating = regiones_validas.groupby('region')['rating'].mean().sort_values(ascending=False).head(10)

region_rating.plot(kind='barh')
plt.title("Regiones con mejor promedio de rating")
plt.xlabel("Rating promedio")
plt.gca().invert_yaxis()
plt.show()




In [ ]:
region_rating = regiones_validas.groupby('region')['rating'].mean()

region_counts = regiones_validas['region'].value_counts()

region_stats = pd.DataFrame({
    'rating_promedio': region_rating,
    'cantidad_vinos': region_counts
}).dropna().sort_values(by='rating_promedio', ascending=False).head(10)

ax = region_stats['rating_promedio'].plot(kind='barh',)
plt.title("Regiones con mejor promedio de rating y cantidad de vinos")
plt.xlabel("Rating promedio")
plt.gca().invert_yaxis()

for i, (rating, cantidad) in enumerate(zip(region_stats['rating_promedio'], region_stats['cantidad_vinos'])):
    plt.text(rating + 0.02, i, f"{int(cantidad)} vinos", va='center')

plt.tight_layout()
plt.show()


### Winery + Rating

---

In [ ]:
bodega_rating = wines_filtrados.groupby('winery')['rating'].mean().sort_values(ascending=False).head(10)
bodega_rating.plot(kind='barh')
plt.title("Bodegas con mejor promedio de rating")
plt.xlabel("Rating promedio")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
plt.style.use("ggplot")
sns.set_palette("viridis")
plt.rcParams["figure.figsize"] = (10,6)

bodega_rating = wines_filtrados.groupby('winery')['rating'].mean()
bodega_top10 = bodega_rating.sort_values(ascending=False).head(10)

bodega_counts = wines_filtrados['winery'].value_counts()
bodega_counts_top10 = bodega_counts[bodega_top10.index]


ax = bodega_top10.plot(kind='barh')
plt.title("Bodegas con mejor promedio de rating")
plt.xlabel("Rating promedio")
plt.gca().invert_yaxis()


for i, (rating, cantidad) in enumerate(zip(bodega_top10, bodega_counts_top10)):
    plt.text(rating + 0.02, i, f"{int(cantidad)} vinos", va='center')

plt.tight_layout()
plt.show()


In [ ]:
top_vinos = wines_filtrados.sort_values(by='rating', ascending=False).head(50)
top_vinos[['name', 'rating', 'price', 'winery']].head(10)


### Style + Rating

---

In [ ]:
wines_filtrados_style = wines.dropna(subset=['style', 'rating'])
style_rating = wines_filtrados_style.groupby('style')['rating'].mean().sort_values(ascending=False).head(10)


plt.figure(figsize=(10, 6))
style_rating.plot(kind='barh')
plt.title("Estilos de vino con mejor rating promedio")
plt.xlabel("Rating promedio")
plt.gca().invert_yaxis()
plt.show()
